In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.misc import imresize
from scipy.optimize import minimize
from sklearn.metrics import classification_report

from PIL import Image, ImageDraw, ImageFont, ImageFilter

In [2]:
GRID_SIZE = (11, 11)

In [3]:
def prepare_image(symbol, modifier=None):
    im = Image.new ("P", (100, 100), (0,))
    draw = ImageDraw.Draw(im)
    font_name = "DejaVuSansMono.ttf"
    if modifier == 'bold':
        font_name = "DejaVuSansMono-Bold.ttf"
    if modifier == 'italic':
        font_name = "DejaVuSerif-Italic.ttf"
    if modifier == 'oblique':
        font_name = "DejaVuSansMono-Oblique.ttf"
    if modifier == 'different':
        font_name = "DejaVuMathTeXGyre.ttf"
    unicode_font = ImageFont.truetype(font_name, 36)
    draw.text((10,10), symbol, font=unicode_font, fill=255)
    im = np.array(im)
    
    ver, hor = (np.where(im > 0))
    im = im[ver.min():ver.max(), hor.min():hor.max()]
    
    im = imresize(im, GRID_SIZE, interp='nearest')
    
    return im

In [4]:
symbols = 'Z S E F V > δ 2 6 8'.split(' ')

train_data = [
    prepare_image(symbols[0]),
    prepare_image(symbols[2]),
    prepare_image(symbols[6]),
    prepare_image(symbols[8]),
    prepare_image(symbols[1]),
    prepare_image(symbols[3]),
    prepare_image(symbols[5]),
    prepare_image(symbols[7]),
    prepare_image(symbols[1]),
    prepare_image(symbols[4]),
    prepare_image(symbols[5]),
    prepare_image(symbols[0]),
    prepare_image(symbols[3]),
    prepare_image(symbols[6]),
    prepare_image(symbols[9]),
    prepare_image(symbols[9]),
]

test_data = [
    prepare_image(symbols[0], 'oblique'),
    prepare_image(symbols[2], 'oblique'),
    prepare_image(symbols[6], 'oblique'),
    prepare_image(symbols[8], 'oblique'),
    prepare_image(symbols[1], 'italic'),
    prepare_image(symbols[3], 'italic'),
    prepare_image(symbols[5], 'italic'),
    prepare_image(symbols[7], 'italic'),
    prepare_image(symbols[1], 'bold'),
    prepare_image(symbols[4], 'bold'),
    prepare_image(symbols[5], 'bold'),
    prepare_image(symbols[0], 'different'),
    prepare_image(symbols[3], 'different'),
    prepare_image(symbols[6], 'different'),
    prepare_image(symbols[9], 'different'),
    prepare_image(str(int(symbols[9]) + 1)),
]

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


OSError: cannot open resource

In [ ]:
def get_feature_response(X, feature):
    return np.array([(x * feature).sum() for x in X])

features = list()
for i in range(GRID_SIZE[0]):
    features.append(np.zeros(GRID_SIZE))
    features[-1][i, :] = 1
    
for i in range(GRID_SIZE[1]):
    features.append(np.zeros(GRID_SIZE))
    features[-1][:, i] = 1
    
for i in range(GRID_SIZE[0]):
    features.append(np.zeros(GRID_SIZE))
    features[-1][i//2:-i//2, i//2:-i//2] = 1
    
features.append(np.eye(GRID_SIZE[0]))
features.append(np.eye(GRID_SIZE[0])[::-1])

In [ ]:
train_responses = list()
for f in features:
    train_responses.append(get_feature_response(train_data, f))
train_responses = np.array(train_responses).T

test_responses = list()
for f in features:
    test_responses.append(get_feature_response(test_data, f))
test_responses = np.array(test_responses).T

In [ ]:
def maxmin_diffs(x, y, weights):
    dists = np.abs(np.expand_dims(x, 1) - np.expand_dims(y, 0)).astype(np.float64)
    dists_weighted = dists * weights.reshape(1, 1, -1)
    
    return dists_weighted.max(axis=2)

def linear_diffs(x, y, weights):
    dists = np.abs(np.expand_dims(x, 1) - np.expand_dims(y, 0)).astype(np.float64)
    dists_weighted = dists * weights.reshape(1, 1, -1)
    
    return dists_weighted.sum(axis=2)

initial_weights = np.ones(len(features))

initial_maxmin_diffs = maxmin_diffs(train_responses, test_responses, initial_weights)
initial_linear_diffs = linear_diffs(train_responses, test_responses, initial_weights)

def norm(x, axis=1):
    return -(x / x.mean())

def softmax(x, axis=1):
    exp = np.exp(x)
    return exp / exp.sum(axis)

def loss(diff_method):

    one_hot_labels = np.eye(test_responses.shape[0])
    
    def _loss(w):
        one_hot_logits = diff_method(train_responses, test_responses, w)
        one_hot_logits = softmax(norm(one_hot_logits))

        return (-(one_hot_labels * np.log(one_hot_logits)).mean(axis=1)).sum()
    
    return _loss


print('Метод максиминой свертки:')
print('Начальная точность %.2f' % (initial_maxmin_diffs.argmin(axis=1) == np.arange(test_responses.shape[0])).mean())

res = minimize(loss(maxmin_diffs), initial_weights)
final_maxmin_weights, fun = res.x, res.fun

final_maxmin_diffs = maxmin_diffs(train_responses, test_responses, final_maxmin_weights)
print('Конечная точность %.2f' % (final_maxmin_diffs.argmin(axis=1) == np.arange(test_responses.shape[0])).mean())

print()

print('Метод линейной свертки:')
print('Начальная точность %.2f' % (initial_linear_diffs.argmin(axis=1) == np.arange(test_responses.shape[0])).mean())

res = minimize(loss(linear_diffs), initial_weights)
final_linear_diffs_weights, fun = res.x, res.fun

final_linear_diffs = linear_diffs(train_responses, test_responses, final_linear_diffs_weights)
print('Конечная точность %.2f' % (final_maxmin_diffs.argmin(axis=1) == np.arange(test_responses.shape[0])).mean())